# Small Experiment Data Analysis

In [1]:
import json
from six.moves import cPickle
import os
import statistics

from IPython.core.display import HTML 
from IPython.core.display import Image, display

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [13]:
"""
Load Coco data
"""
COCO_PATH = 'data/coco/'
COCO_ANNO_PATH = COCO_PATH + 'annotations/'

# load the data from captions
with open(COCO_ANNO_PATH + 'captions_train2014.json') as f:
    coco_captions = json.load(f)
# print len(coco_captions)
# with open(COCO_ANNO_PATH + 'captions_val2014.json') as f:
#     coco_captions =  dict(coco_captions.items() + json.load(f).items())
# print len(coco_captions)

# build the reverse dictionary, from img_id to captions, img_infos, and annotations
img_captions = {}
for img_info in coco_captions['images']:
    mid = int(img_info['id'])
    if not mid in img_captions:
        img_captions[mid] = {}
    img_captions[mid]['image'] = img_info

for cap_info in coco_captions['annotations']:
    mid = int(cap_info['image_id'])
    if not 'annotation' in img_captions[mid]:
        img_captions[mid]['annotation'] = []
        img_captions[mid]['captions'] = []
    img_captions[mid]['annotation'].append(cap_info)


In [17]:
IN_FILE_NAME  = 'high_coexist_img_ids.pkl'
OUT_FILE_NAME = 'highexist_gaussian'
IN_DIR        = 'input'
OUT_DIR       = 'output'

def get_stat(data):
    if len(data)==0: return;
    print ""
    print "Total:\t%d;\t\tMean:\t\t%f;\tNonezero:\t%d"%(len(data), statistics.mean(data), len(filter(lambda x:x!=0, data)))
    print "Median:\t%f;\tMedian(H):\t%f;\tMedian(L):\t%f;"%(statistics.median(data), statistics.median_high(data), statistics.median_low(data))
    print "Max:\t%f;\tMin:\t\t%f;\tStd:\t\t%f"%(max(data), min(data),statistics.stdev(data))
    print "\n"
    
def look_up_image(title, idx, in_ids, vis, scores):
    print "[%d]%s\n"%(idx,title)
    print "\tScore:%s;\tImageId:%s\n"%(scores[idx], in_ids[idx])
    print "\tOriginal Caption:\n\t\t%s;\n\tAblation Caption:\n\t\t%s;"%(vis[idx*2+1]['caption'],vis[idx*2]['caption'])
    print "\tAnnotated Captions:\n"
    img_id = int(in_ids[idx])
    for idx, note in enumerate(img_captions[img_id]['annotation']):
        print "\t\t%d. %s\n"%(idx+1, note['caption'])
    display(Image(url= img_captions[img_id]['image']['coco_url']))

def get_expr_summary(in_file=IN_FILE_NAME, out_file=OUT_FILE_NAME, in_path=IN_DIR, out_path=OUT_DIR, num=5):
    in_ids  = cPickle.load(open(os.path.join(in_path, in_file)))
    scores  = cPickle.load(open(os.path.join(out_path, "scores_%s.pkl"%out_file)))
    vis     = json.load(open(os.path.join(out_path, 'vis_%s.json'%out_file)))
    print "Statistics for scores of gaussian filtered ablations:"
    get_stat(scores)

    scores_idx = zip(range(len(scores)), scores)
    sorted_scores_idx = sorted(scores_idx, key=lambda x: x[1])   # sort by score
    for idx, _ in sorted_scores_idx[:num]:
        look_up_image("Zero Image",idx, in_ids, vis, scores)



In [18]:
IN_FILE_NAME  = 'high_coexist_img_ids.pkl'
OUT_FILE_NAME = 'highexist_gaussian'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	1013;		Mean:		0.837452;	Nonezero:	1008
Median:	1.000000;	Median(H):	1.000000;	Median(L):	1.000000;
Max:	1.000000;	Min:		0.000000;	Std:		0.276086


[107]Zero Image

	Score:0.0;	ImageId:340096

	Original Caption:
		a stop sign is on the corner of a street;
	Ablation Caption:
		a person holding a glass of wine in a room;
	Annotated Captions:

		1. A stop sign covered in lights at a street intersection.

		2. A stop sign that is decorated with Christmas lights.

		3. a decorated lite up stop sign at the corner

		4. car comes to the curve to stop at the stop sign

		5. A stop sign covered in a white lights. 



[427]Zero Image

	Score:0.0;	ImageId:110251

	Original Caption:
		a street sign on a street corner;
	Ablation Caption:
		a park bench sitting in the middle of a park;
	Annotated Captions:

		1. A bench sitting on top of a park near a play ground.

		2. a fenced in park on a city street

		3. a sign showing that there kids playing in the field

		4. a sign on a pole near a park 

		5. A playground with a sign saying caution live children playing.



[550]Zero Image

	Score:0.0;	ImageId:289515

	Original Caption:
		a street sign on the side of a street;
	Ablation Caption:
		a car parked in front of a building;
	Annotated Captions:

		1. A sidewalk and doughnut shop with a bike leaning against the building

		2. A long shot of a city sidewalk on a sunny day 

		3. A sign for a doughnut shop on a sidewalk by a street.

		4. Donut shop sign on an empty street next to a tree

		5. A building with a sign that says Donuts above the door. 



[584]Zero Image

	Score:0.0;	ImageId:461464

	Original Caption:
		a clock on the side of a building;
	Ablation Caption:
		a train station with a train passing by;
	Annotated Captions:

		1. A train station has a foreign sign and a clock.

		2. View from the train at Pistoia station in Italy

		3. A clock on a train platform during day time.

		4. a clock attached to a pole at a train station

		5. A street sign and a clock sitting by a parked train.



[877]Zero Image

	Score:0.0;	ImageId:199678

	Original Caption:
		a bunch of different types of electronic devices;
	Ablation Caption:
		a pair of scissors sitting on top of a table;
	Annotated Captions:

		1. A stack of cash sitting under a cell phone.

		2. USA 20 dollars totaling $120, held down by a cell phone with Coca cola cans nearby. 

		3. A bunch of twentys underneath a celll phone

		4. Six twenty dollar bills are underneath a phone.

		5. A number of twenty dollar bills with a phone



In [19]:
IN_FILE_NAME  = 'high_coexist_img_ids.pkl'
OUT_FILE_NAME = 'highexist_blackout'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	1013;		Mean:		0.652322;	Nonezero:	976
Median:	0.714286;	Median(H):	0.714286;	Median(L):	0.714286;
Max:	1.000000;	Min:		0.000000;	Std:		0.348214


[30]Zero Image

	Score:0.0;	ImageId:178533

	Original Caption:
		an airplane is parked on a runway in a field;
	Ablation Caption:
		a group of people standing around a plane;
	Annotated Captions:

		1. A group of pilots posing in front of a fighter jet.

		2. A fighter jet parked behind U.S. Air Force pilots.

		3. Air Force member posed in front of a jet 

		4. a group of people kneeling down next to an air force jet plane

		5. A group of pilots posing in front of a jet.



[62]Zero Image

	Score:0.0;	ImageId:8379

	Original Caption:
		a street sign on the side of the road;
	Ablation Caption:
		a tv sitting in the middle of a forest;
	Annotated Captions:

		1. An old "Parking" sign is attached to a rusted pole.

		2. A PICTURE OF A PARKING SIGN ON THE STREET 

		3. An old rusty pole that has a sign "Parking" hanging from it. Across from it is an old brick building with three windows.

		4. a pole with a sign underneath a tree and in front of a building

		5. a parking sighn on a feild next to a house. 



[63]Zero Image

	Score:0.0;	ImageId:466941

	Original Caption:
		a sign that says UNK UNK on the side of it;
	Ablation Caption:
		a fire hydrant in the middle of a forest;
	Annotated Captions:

		1. A tall sign advertising Cider in front of a net.

		2. A large sign is advertising cold fresh cider.

		3. An apple cider sign between colorful Autumn foliage.

		4. A painted sign for cold, fresh cider sits outside.

		5. This is the place to stop for cold cider on a hot day.



[86]Zero Image

	Score:0.0;	ImageId:352828

	Original Caption:
		a stop sign on the side of the road;
	Ablation Caption:
		a car parked in front of a building;
	Annotated Captions:

		1. An old two story house next to a forest.

		2. a street sign on a pole with buildings in the background 

		3. A house that is sitting next to trees.

		4. A stop sign across the road from a weathered building.

		5. A stop sign is shown, with an abandoned house.



[130]Zero Image

	Score:0.0;	ImageId:504524

	Original Caption:
		a street sign on a pole in a wooded area;
	Ablation Caption:
		a black and white photo of a parking meter;
	Annotated Captions:

		1. a close up of a vandalized street sign

		2. A wrong way sign under a do not enter sign.

		3. Black and white warning signs on roadway pole.

		4. Black and white photograph of two signs that read "do not enter" and "wrong way."

		5. a very big sign showing that no way



In [20]:
IN_FILE_NAME  = 'large_text_img_ids.pkl'
OUT_FILE_NAME = 'large_text_blackout'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)

Statistics for scores of gaussian filtered ablations:

Total:	683;		Mean:		0.521510;	Nonezero:	630
Median:	0.500000;	Median(H):	0.500000;	Median(L):	0.500000;
Max:	1.000000;	Min:		0.000000;	Std:		0.352480


[23]Zero Image

	Score:0.0;	ImageId:330436

	Original Caption:
		a man is doing a trick on a skateboard;
	Ablation Caption:
		a person jumping a skate board in the air;
	Annotated Captions:

		1. a person jumping a skate board in the air

		2. This skateboarder is riding the side of his skateboard in the air.

		3. The young man is riding his skateboard practicing his moves. 

		4. A skateboarder is airborne in front of a wall covered in graffiti.

		5. A person jumping in the air on a skateboard.



[27]Zero Image

	Score:0.0;	ImageId:375294

	Original Caption:
		a couple of cell phones sitting on top of a table;
	Ablation Caption:
		a person is holding a small piece of paper;
	Annotated Captions:

		1. there are 2 tablets, 1 e-reader, and an iphone sitting on the table.

		2. a table that has a bunch of ipads on it

		3. some different tablets cellphone and a piece of pizza

		4. Tablets and a cell phone on a table beside a plate of pizza.

		5. Some iPads and phone are on a table with pizza.



[33]Zero Image

	Score:0.0;	ImageId:172316

	Original Caption:
		a young boy swinging a baseball bat at a ball;
	Ablation Caption:
		a man standing in front of a wall with a cell phone;
	Annotated Captions:

		1. A boy is swinging at a ball during a baseball game.

		2. A boy that is hitting a ball with a baseball bat.

		3. A person with a bat and helmet hits a ball.

		4. a boy in a yellow and white uniform hitting a baseball

		5. A young baseball player is hitting the ball. 



[36]Zero Image

	Score:0.0;	ImageId:52943

	Original Caption:
		a group of men playing a game of soccer;
	Ablation Caption:
		a baseball player pitching a ball on top of a field;
	Annotated Captions:

		1. Grown men playing an indoor soccer game on turf.

		2. A man kicking a soccer ball on a field as others run by.

		3. A man is playing soccer during a match. 

		4. some people playing soccer while a crowd watches them 

		5. An individual is taken in this very picture. 




[42]Zero Image

	Score:0.0;	ImageId:531831

	Original Caption:
		a man standing in front of a store window;
	Ablation Caption:
		a living room with a tv and a tv;
	Annotated Captions:

		1. An eight book boxed set collection of a story.

		2. A set of books that read "TINTIN" on the side of the box set.

		3. A  boxed set of books with pictures on them

		4. A book set of the adventures of tintin.

		5. a close up of a set of books



In [ ]:
IN_FILE_NAME  = 'large_text_img_ids.pkl'
OUT_FILE_NAME = 'large_text_blackout'
get_expr_summary(in_file = IN_FILE_NAME, out_file = OUT_FILE_NAME)